# 一、 图卷积网络 (GCN) 中的归一化处理知识点整理

## 1. 核心概念与问题背景

### 1.1 图数据表示
- **图**：由节点（Nodes）和边（Edges）构成，常用邻接矩阵（Adjacency Matrix $A$） 表示连接关系。
- **节点特征**：每个节点可拥有特征向量（Feature vector $X$），用于描述节点属性。
- **度矩阵（Degree Matrix）** $D$：对角矩阵，对角线元素 $D_{ii}$ 表示节点 $i$ 的度（相连边的数量）。

![](assets/01-01.png)

### 1.2 核心问题
直接使用邻接矩阵 $A$ 与特征 $X$ 进行运算 ($AX$) 存在两个问题：
1. **未考虑自身信息**：$A$ 中对角线元素为0，运算时未包含节点自身特征。
2. **未归一化**：节点的度不同，导致特征聚合后尺度不一致（度大的节点特征值过大）。

## 2. 解决方案：归一化与公式演化

### 2.1 添加自环 (Self-loop)
- **方法**：在原邻接矩阵 $A$ 基础上加上单位矩阵 $I$（乘以一个系数 $\lambda$，通常取1），确保每个节点在信息聚合时也能考虑到自身的特征。
- **公式**：$\tilde{A} = A + \lambda I_N$

### 2.2 构造新的度矩阵
- **方法**：基于添加自环后的新邻接矩阵 $\tilde{A}$ 计算度矩阵，记录了每个节点（包含自环）的总连接数，为后续的归一化操作提供基准。
- **公式**：$\tilde{D} = D + I_N$

### 2.3 行归一化 (Row Normalization)
- **方法**：使用新的度矩阵 $\tilde{D}$（$\tilde{A}$ 的度矩阵）的逆 $\tilde{D}^{-1}$ 左乘 $\tilde{A}$。
- **公式**：$\tilde{D}^{-1}\tilde{A}X$

### 2.4 列归一化 (Column Normalization)
- **方法**：使用新的度矩阵 $\tilde{D}$（$\tilde{A}$ 的度矩阵）的逆 $\tilde{D}^{-1}$ 右乘 $\tilde{A}$。
- **公式**：$\tilde{A}\tilde{D}^{-1}X$

### 2.5 对称归一化 (Symmetric Normalization)
- **问题**：仅进行行归一化（对行操作）可能仍不够均衡，使用度矩阵的 $-1/2$ 次幂进行左右乘法，实现对称归一化。
- **公式**：$\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} X$
- **效果**：这种对称归一化方法在GCN中更为常用，能产生更稳定和有效的节点表示。

## 3. 关键矩阵定义与计算

| 矩阵符号 | 名称 | 定义与计算 | 作用 |
| :--- | :--- | :--- | :--- |
| $A$ | 邻接矩阵 | 表示节点间的连接关系，$A_{ij}=1$ 表示节点 i, j 相连 | 基础连接关系 |
| $I_N$ | 单位矩阵 | 主对角线为1，其余为0的方阵 | 用于添加自环 |
| $\tilde{A}$ | 添加自环后的邻接矩阵 | $\tilde{A} = A + I_N$ (假设 $\lambda=1$) | 使节点聚合包含自身信息 |
| $\tilde{D}$ | 新度矩阵 | $\tilde{D}$ 是 $\tilde{A}$ 的度矩阵，是对角阵，$\tilde{D}_{ii} = \sum_{j} \tilde{A}_{ij}$ | 计算归一化系数 |
| $\tilde{D}^{-1}$ | 度矩阵的逆 | $\tilde{D}^{-1}$ 对角线元素为 $1/\tilde{D}_{ii}$ | 用于行/列归一化 |
| $\tilde{D}^{-1/2}$ | 度矩阵的 $-1/2$ 次幂 | $\tilde{D}^{-1/2}$ 对角线元素为 $1/\sqrt{\tilde{D}_{ii}}$ | 用于对称归一化 |

## 4. 总结

图卷积网络的两层核心操作可表示为：

$$Z = f(X, A) = \text{softmax}\left(\hat{A}\ \text{ReLU}\left(\hat{A}XW^{(0)}\right)W^{(1)}\right)$$

各组成部分的维度分别为：
- $X$：节点特征矩阵（维度 $N \times C$）
- $W^{(0)}$：第一层可训练权重矩阵（维度 $C \times H$）
- $W^{(1)}$：第二层可训练权重矩阵（维度 $H \times F$）
- $\hat{A}$：对称归一化后的邻接矩阵（维度 $N \times N$）

| 符号 | 含义 |
|------|------|
| **N** | 节点数量 (Number of nodes) |
| **C** | 输入特征维度 (Input feature dimension) |
| **H** | 隐藏层维度 (Hidden layer dimension) |
| **F** | 输出维度 (Output dimension) |

该公式清晰地体现了GCN的两层传播结构：
- 首先通过 $\hat{A}XW^{(0)}$ 进行第一层特征变换和邻居聚合
- 经ReLU激活函数非线性变换
- 再次通过 $\hat{A}(\cdot)W^{(1)}$ 进行第二层传播和特征变换
- 最终通过softmax函数输出分类结果

# 二、 节点分类

![](assets/01-02.png)

# 三、 边预测

![](assets/01-03.png)

![](assets/01-04.png)